## pickle格式
python内置的pickle序列化是最简单的实现数据高效二进制格式存储方法之一。pandas每个对象都有`to_pickle`方法将数据以pickle格式保存到磁盘上。
> 注：pickle仅建议用于短期存储格式，因为后续版本很难保证格式的稳定，现在生成的pickle格式文件有可能无法被后续版本读取。

In [1]:
import pandas as pd

In [2]:
frame=pd.read_csv('doc\csv-example.csv')
frame

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


In [3]:
frame.to_pickle('doc\pickle-example') # 以pickle格式保存
!dir doc\pickle-example

 驱动器 E 中的卷是 新加卷
 卷的序列号是 5023-9879

 E:\jupyter-notebook\doc 的目录

2019/02/18  22:23               986 pickle-example
               1 个文件            986 字节
               0 个目录 45,307,654,144 可用字节


In [4]:
pd.read_pickle('doc\pickle-example') # 读取pickle格式文件

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


## HDF5格式
HDF5是一种存储大规模科学数组数据的非常好的文件格式。每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，HDF5支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写。  
虽然可以用`PyTables`或`h5py`库直接访问HDF5文件，pandas提供了更为高级的接口，可以像字典一样存储Series和DataFrame对象。 
  
**常用方法**  
`HDFStore`：打开HDF5文件，建立HDFStore对象  
参数：  
`mode`：
- a（默认，打开文件等待读写，若文件不存在则先新建）
- w（新建一个文件，若已存在则删除再新建）
- r（只允许读取）
- r+（与a相似，但文件必须存在）  
  
`put`：将数据存入HDF5文件，支持`fixed`和`table`两种存储模式  
参数：  
`format`：
- `fixed`：默认，快速读写，存入的数据不能添加，不能查询，与store['obj']字典方式相同
- `table`：速度慢，但存入的数据支持使用特殊语法进行查询操作
  
`select`：从HDF5文件中读取数据
参数：
`where`：支持查询操作  
  
`to_hdf`、`read_hdf`：不使用HDFStore方法，直接操作HDF5文件，方法类似`put`和`select`，但要指定HDF5文件名

In [5]:
store=pd.HDFStore('doc\mydata.h5',mode='w') # 建立、打开HDF5文件，建立对象，mode='w'若文件存在先删除
store['obj']=frame # 以字典方式存入数据
store['obj_col']=frame['A'] # 存入Series
print(store.info()) # 查看HDF5文件信息

<class 'pandas.io.pytables.HDFStore'>
File path: doc\mydata.h5
/obj                frame        (shape->[4,6])
/obj_col            series       (shape->[4])  


In [6]:
store['obj'] # 读取数据

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


In [7]:
store.put('obj1',frame,format='table') # 以table存储模式存入数据
store['obj1'] # 以字典方式读取

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


In [8]:
store.select('obj1',where=['index>=2 and columns in ["A","C"]']) # select方法支持查询

,A,C
2,11,13
3,16,18


In [9]:
store.close() # HDFStore对象需要关闭释放资源

In [10]:
frame.to_hdf('doc\mydata.h5','obj3',format='table') # 直接写文件
pd.read_hdf('doc\mydata.h5','obj3',where=['index>=2 and columns in ["A","C"]']) # 直接读取

,A,C
2,11,13
3,16,18


## Excel格式
将数据写入Excel：先创建`ExcelWriter`，然后使用`to_excel`将数据写入，最后`save`保存。也可以不使用`ExcelWriter`，直接传递文件路径到`to_excel`，但这样只能创建一个表单，`ExcelWriter`可以多次调用`to_excel`来创建多个表单。

In [11]:
writer=pd.ExcelWriter('doc\excel-example.xlsx')
frame.to_excel(writer,'表单A') # 默认表单名为Sheet1
frame.to_excel(writer,'表单B') # 可以创建多个表单
writer.save()

In [12]:
frame.to_excel('doc\excel-example.xlsx','表单A') # 直接创建文件并写入数据，但只能创建一个表单

读取Excel：先通过路径创建`ExcelFile`实例，然后使用`read_excel`读取excel内容到DataFrame。也可以直接传递文件路径到`read_excel`来读取excel内容，但若要读取多个表单则要多次读取文件，而创建`ExcelFile`只需要读取一次文件然后对实例进行操作，速度更快。

In [13]:
reader=pd.ExcelFile('doc/excel-example.xlsx')
pd.read_excel(reader,'表单A') # 不写表单名的话默认读取第一个表单

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


In [14]:
pd.read_excel('doc/excel-example.xlsx') # 直接传入路径并读取数据

,A,B,C,D,E,备注
0,1,2,3,4,5,a
1,6,7,8,9,10,b
2,11,12,13,14,15,c
3,16,17,18,19,20,d


In [15]:
reader.close()

> 技巧：无论`ExcelWriter`还是`to_excel`方法都无法向已经存在的文件中添加新的表单，我们可以通过openpyxl库来辅助实现数据的添加新表单

In [16]:
from openpyxl import load_workbook
excelWriter = pd.ExcelWriter('doc\excel-example.xlsx',engine='openpyxl') # 必须指定解析引擎engine='openpyxl'

book = load_workbook(excelWriter.path)
excelWriter.book = book
frame[['B','D','E']].to_excel(excelWriter,'表单C') # 将DataFrame写入新表单
excelWriter.close()

pd.read_excel('doc/excel-example.xlsx','表单C') # 读取表单C进行验证

,B,D,E
0,2,4,5
1,7,9,10
2,12,14,15
3,17,19,20
